# First attempt of developing web crawlers

        !---------- Reuters AGB & Scraping -----------!
Die Nutzungsbedingungen von Thomson Reuters und Reuters legen klare Beschränkungen bezüglich der Verwendung ihrer Inhalte fest, die gegen das Auslesen von Artikeln mittels Webscraping sprechen würden. Hier eine Zusammenfassung der relevanten Punkte in Markdown-Format:

### Beschränkungen gemäß den Nutzungsbedingungen von Thomson Reuters:

- **Lizenz**: Nutzern wird ein weltweites, nicht ausschließliches, nicht übertragbares, widerrufbares, begrenztes Recht zur Zugriff und Nutzung der Informationen und Dienste auf der Website gewährt, strikt gemäß diesen Bedingungen. Jede Nutzung, die über die Beschreibung von Thomson Reuters Produkten und Dienstleistungen zur Bildung von Nutzern oder potenziellen Nutzern hinausgeht, erfordert die vorherige schriftliche Zustimmung von Thomson Reuters【8†source】.

- **Verbotene Nutzung**: Die Website darf nicht in einer Weise genutzt werden, die:
  - Schaden anrichten könnte, einschließlich der Server, Netzwerke und anderer Komponenten, die für die Website genutzt werden.
  - Versuche beinhaltet, unbefugten Zugriff zu erlangen zu Diensten, Konten, Computersystemen oder Netzwerken, die mit irgendeinem Server verbunden sind, der für die Website verwendet wird【8†source】.

- **Weiterverbreitung und Kopieren**: Ohne ausdrückliche schriftliche Zustimmung von Thomson Reuters ist es nicht gestattet, Inhalte zu reproduzieren, zu duplizieren, zu kopieren, herunterzuladen, zu speichern, weiter zu übertragen, zu verbreiten, zu transferieren oder anderweitig zu nutzen【8†source】.

### Fazit:
Auf Grundlage dieser Bedingungen scheint Webscraping ohne ausdrückliche Zustimmung von Thomson Reuters gegen deren Nutzungsbedingungen zu verstoßen und könnte rechtliche Konsequenzen nach sich ziehen. Es ist empfehlenswert, direkt Kontakt mit Thomson Reuters aufzunehmen, um eine ausdrückliche Erlaubnis oder Lizenz für solche Aktivitäten zu erhalten, falls dies von Interesse ist.

Für detailliertere Informationen besuchen Sie bitte direkt die [Thomson Reuters Nutzungsbedingungen](https://legal.thomsonreuters.com/en/products/terms-of-use).

    !--------------------------------------------------------!

# Crawler

### 1. Reuters search fuction with "Boeing" as input

Structure of Reuters search domain:

https://www.reuters.com/site-search/?query=boeing&offset=20

                                    ↑         ↑
                           search string     number of articles displayed in the list (multiple of 20)
                                        

### Crawler für Links zu den einzelnen Artikeln

Es wird das gegebene Wort (searchString) in die Suchleiste bei Reuters eingegeben. Danach wird untersucht, wieviele Artikel es zu diesem Thema gibt. Auf basis dieser Anzahl werden die Anzeigeseiten durchgegangen um die jeweils 20 angezeigten Links zu den Artikeln zu bekommen. Diese werden in einer Liste gespeichert.

Aufbau der Suchergebnisse: https://www.reuters.com/site-search/?query=boeing&offset=20

In [27]:
# Benötigte Pakete
! pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Safari()
searchString= "boeing"

# Liste für die Links erstellen
links_list = []

# WebDriver für Safari initialisieren


try:
    # Webseite öffnen
    driver.get(f'https://www.reuters.com/site-search/?query={searchString}&offset=20')
    
    # Warten, um sicherzustellen, dass die Seite vollständig geladen ist
    time.sleep(2)
    
    # Element mit der Anzahl der Artikel finden 
    element = driver.find_element(By.CSS_SELECTOR, 'span.search-results__text__13FtQ')
    
    # Gesamtzahl der Artikel extrahieren und berechnen, wie viele Seiten besucht werden müssen
    total_articles = int(element.text.split()[-1])
    pages = -(-total_articles // 20)  # Aufrunden der Division

    for page in range(1, pages):
        
        if page > 2:  # Bei der ersten Seite sind wir bereits, keine Notwendigkeit die Seite neu zu laden
            driver.get(f'https://www.reuters.com/site-search/?query={searchString}&offset={page * 20}')
            time.sleep(5)
        
        # Alle Blöcke mit der angegebenen Klasse finden
        items = driver.find_elements(By.CSS_SELECTOR, 'li.search-results__item__2oqiX')
        for item in items:
            # Innerhalb jedes Items nach dem Link-Element suchen
            link_element = item.find_element(By.CSS_SELECTOR, 'div.basic-card__container__2c0tk.default')
            link = link_element.get_attribute('href')
            links_list.append(link)
        print(f'Es wird Seite {page} von {pages} auf Links von Artikeln untersucht!')


finally:
    # Browser schließen
    driver.quit()

# Doppelt genannte Artikel entfernen
unique_links_list = list(set(links_list))

# Ausgabe der gesammelten Links 
print("Fertig! Die Variable unique_links_lsit enthält alle Links zu passenden Artikeln.")


Es wird Seite 1 von 132 auf Links von Artikeln untersucht!
Es wird Seite 2 von 132 auf Links von Artikeln untersucht!
Es wird Seite 3 von 132 auf Links von Artikeln untersucht!
Es wird Seite 4 von 132 auf Links von Artikeln untersucht!


KeyboardInterrupt: 

### Die gesammelten Links in einer CSV speichern

In [50]:
import csv

# Angenommen, links_list ist deine Liste mit Links


# Pfad zur CSV-Datei, die du erstellen möchtest
file_path = 'gesammelte_links.csv'

# CSV-Datei öffnen und Links schreiben
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Für jeden Link eine Zeile in der CSV-Datei erstellen
    for link in unique_links_list:
        writer.writerow([f"https://www.reuters.com{link}"])

print(f"Links wurden erfolgreich in {file_path} gespeichert.")


Links wurden erfolgreich in gesammelte_links.csv gespeichert.


### Crawler für jeden einzelnen Artikel und dessen Content 

Link zum Beispielartikel:
https://www.reuters.com/business/aerospace-defense/airbus-open-second-china-assembly-line-finalises-jet-order-2023-04-06/

In [15]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.by import By
'''
#________________________________________________________________________
# Nur wenn Firefox verwendet werden soll
from selenium.webdriver.firefox.options import Options as FirefoxOptions

# Benutzerdefinierten User-Agent String festlegen
CUSTOM_USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"

# FirefoxOptions instanzieren
options = FirefoxOptions()

# Den User-Agent im Firefox durch Options ändern
options.set_preference("general.useragent.override", CUSTOM_USER_AGENT)
#options.add_argument("--headless")

# Für lokale WebDriver-Sitzungen, Firefox direkt instanziieren:
driver = webdriver.Firefox(options=options)
#________________________________________________________________________
'''

#_____________________________________________
# Nur wenn Safari verwendet werden soll
driver = webdriver.Safari()
#_____________________________________________


# URL des Artikels
url = "https://www.reuters.com/business/aerospace-defense/airbus-open-second-china-assembly-line-finalises-jet-order-2023-04-06/"

# Öffne die Webseite
driver.get(url)

# Warte einen Moment, um sicherzustellen, dass die Seite geladen ist
driver.implicitly_wait(5)


# Autor extrahieren
try:
    # Versuche, das Element mit dem ersten Selector zu finden
    autor_element = driver.find_element(By.CSS_SELECTOR, '[rel="author"]')
except NoSuchElementException:
    # Wenn das Element nicht gefunden wurde, versuche es mit dem zweiten Selector
    try:
        autor_element = driver.find_element(By.CSS_SELECTOR, '[class="text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__small__1kGq2 info-content__author__1GnSx"]')
    except NoSuchElementException:
        autor = "Autor nicht gefunden"
autor = autor_element.text


# Titel extrahieren
try:
    # Versuche, das Element mit dem ersten Selector zu finden
    titel_element = driver.find_element(By.CSS_SELECTOR, '[class="text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_3__1kDhc heading__base__2T28j heading__heading_3__3aL54"]')
except NoSuchElementException:
    titel = "Titel nicht gefunden"
titel = titel_element.text


# Abteilung / Bereich extrahieren
try:
    bereich_element= driver.find_element(By.CSS_SELECTOR, '[class="text__text__1FZLe text__dark-grey__3Ml43 text__regular__2N1Xr text__extra_small__1Mw6v link__link__3Ji6W link__underline_on_hover__2zGL4 tags-with-tooltip__link__3Mb0Y"]')
except NoSuchElementException:
    bereich = "Bereich nicht gefunden"
bereich = bereich_element.text


# Erscheinungsdatum und Uhrzeit extrahieren
datum_zeit_elemente = driver.find_elements(By.CSS_SELECTOR, 'time[data-testid="Body"] .date-line__date___kNbY')
datum = datum_zeit_elemente[0].text if datum_zeit_elemente else 'Datum nicht gefunden'
zeit = datum_zeit_elemente[1].text if len(datum_zeit_elemente) > 1 else 'Zeit nicht gefunden'


# Text extrahieren
try: # Ein Text ist in mehrere Paragraphen unterteilt
    paragraphs_elements = driver.find_elements(By.CSS_SELECTOR, 'div.article-body__paragraph__2-BtD')
except NoSuchElementException:
    bereich = "Text nicht gefunden"

# Extrahiere den Text aus jedem Paragraphen
text = " ".join([paragraph.text for paragraph in paragraphs_elements])
    

# Ausgabe der extrahierten Informationen
print("Autor:", autor)
print("Titel:", titel)
print("Abteilung:", bereich)
print("Datum:", datum)
print("Zeit:", zeit)
print("Text:", text)



# WebDriver schließen
driver.quit()


Autor: Michel Rose
Titel: Airbus to open new China assembly line amid recovery
Abteilung: Aerospace & Defense
Datum: April 6, 2023
Zeit: 5:54 PM GMT+2
Text: BEIJING/PARIS, April 6 (Reuters) - Europe's Airbus (AIR.PA), opens new tab moved to double capacity in China and strengthen access to the world's second-largest aviation market by agreeing on Thursday to build a second Chinese assembly line, as domestic travel returns to pre-pandemic levels. The world's largest planemaker, which has overtaken Boeing as a supplier to China amid tensions between Washington and Beijing, also got the go-ahead to deliver 160 jets already sold but failed to win new orders during a French state visit. "The recovery here is quite impressive. We see very strong momentum," Airbus Chief Executive Guillaume Faury told reporters during his first visit to China since the COVID-19 crisis. Airbus says China’s traffic will grow by 5.3% annually over the next two decades, outstripping a global average of 3.6%. The F